In [ ]:
import pandas as pd


In [ ]:
#load in data, clean data
df = pd.read_csv('global_cancer_patients_2015_2024.csv')
#remove nan values
df = df.dropna()
df_sampled = df.sample(n=100, random_state=42) #make smaller just for practice
data = df_sampled.to_dict('records')


In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'transformers'

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # model for encoding
qdrant = QdrantClient(":memory:")
qdrant.recreate_collection(
    collection_name="cancer_patients",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)
qdrant.upload_records(
    collection_name="cancer_patients",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["Cancer_Type"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)
hits = qdrant.search(
    collection_name="cancer_patients",
    query_vector=encoder.encode("statistics for a patient with lung cancer").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

search_results = [hit.payload for hit in hits]


In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", 
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "cancer chatbot, give data"},
        {"role": "user", "content": "give me info about patients w/ lung cancer"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)